In [1]:
import os
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
import shap

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


# Preparation of dataset.

In [2]:
def load_excel(path):
    df = pd.read_excel(path)
    return df

In [3]:
def process_dataset(df):
    # Valid data with sd<0.1.
    df = df[df["K_ave_sd"]<0.1]
    
    # Rename columns.
    col_rename = {}
    for col in df.columns[5:-2]:
        col_new = col.replace("log10[","").replace("]\n(mM)","")
        col_rename[col] = col_new
    col_rename["Strain (gene)"] = "Gene"
    df = df.rename(columns = col_rename)
    
    # Remove columns of H (Not considered in this study).
    df = df.drop("H",axis=1)

    return df

In [4]:
# Import data.
raw_data = load_excel(os.getcwd()+"/Data.xlsx")

# Process data.
data = process_dataset(raw_data)

In [5]:
data

Unique ID Combination ID Medium ID Condition ID_unique  Gene  \
0      C00L2-1_btuR     C00L2_btuR     C00L2             C00L2-1  btuR   
1      C00L2-1_cobB     C00L2_cobB     C00L2             C00L2-1  cobB   
2      C00L2-1_mazG     C00L2_mazG     C00L2             C00L2-1  mazG   
3       C00L2-1_mog      C00L2_mog     C00L2             C00L2-1   mog   
4      C00L2-1_ilvH     C00L2_ilvH     C00L2             C00L2-1  ilvH   
...             ...            ...       ...                 ...   ...   
15475  C0138-1_pdxY     C0138_pdxY     C0138             C0138-1  pdxY   
15483  C0138-1_thiF     C0138_thiF     C0138             C0138-1  thiF   
15494  C0138-1_acpT     C0138_acpT     C0138             C0138-1  acpT   
15495  C0138-1_gabD     C0138_gabD     C0138             C0138-1  gabD   
15523  C0138-1_pdxH     C0138_pdxH     C0138             C0138-1  pdxH   

        Glucose   Citrate  Ammonium  Phosphate   Sulfate  ...       VB3  \
0      0.376577 -2.744727  0.514548   1.820595  0.209431  ... -3.267606   
1      0.376577 -2.744727  0.514548   1.820595  0.209431  ... -3.267606   
2      0.376577 -2.744727  0.514548   1.820595  0.209431  ... -3.267606   
3      0.376577 -2.744727  0.514548   1.820595  0.209431  ... -3.267606   
4      0.376577 -2.744727  0.514548   1.820595  0.209431  ... -3.267606   
...         ...       ...       ...        ...       ...  ...       ...   
15475  2.260548 -2.744727  1.968483   2.202856  1.022988  ... -3.267606   
15483  2.260548 -2.744727  1.968483   2.202856  1.022988  ... -3.267606   
15494  2.260548 -2.744727  1.968483   2.202856  1.022988  ... -3.267606   
15495  2.260548 -2.744727  1.968483   2.202856  1.022988  ... -3.267606   
15523  2.260548 -2.744727  1.968483   2.202856  1.022988  ... -3.267606   

            VB5       VB6       VB8       VB9      VB12    Borate      PABA  \
0     -3.899629 -4.829856 -5.899629 -6.443697 -7.744727 -3.142668 -3.267606   
1     -3.899629 -4.829856 -5.899629 -6.443697 -7.744727 -3.142668 -3.267606   
2     -3.899629 -4.829856 -5.899629 -6.443697 -7.744727 -3.142668 -3.267606   
3     -3.899629 -4.829856 -5.899629 -6.443697 -7.744727 -3.142668 -3.267606   
4     -3.899629 -4.829856 -5.899629 -6.443697 -7.744727 -3.142668 -3.267606   
...         ...       ...       ...       ...       ...       ...       ...   
15475 -5.899629 -6.744727 -7.899629 -3.443697 -6.045757 -1.443697 -0.267606   
15483 -5.899629 -6.744727 -7.899629 -3.443697 -6.045757 -1.443697 -0.267606   
15494 -5.899629 -6.744727 -7.899629 -3.443697 -6.045757 -1.443697 -0.267606   
15495 -5.899629 -6.744727 -7.899629 -3.443697 -6.045757 -1.443697 -0.267606   
15523 -5.899629 -6.744727 -7.899629 -3.443697 -6.045757 -1.443697 -0.267606   

         K_ave  K_ave_sd  
0      0.11000  0.018129  
1      0.11850  0.017078  
2      0.10750  0.000707  
3      0.14500  0.001414  
4      0.11975  0.007632  
...        ...       ...  
15475  0.28450  0.024749  
15483  0.32100  0.038184  
15494  0.19150  0.036062  
15495  0.33800  0.036770  
15523  0.48350  0.054447  

[13944 rows x 52 columns]

# Perform GBDT for the all dataset.

In [6]:
def load_csv(path):
    df = pd.read_csv(path, index_col=0)
    return df

In [7]:
# Perform GBDT and calculate feature importances.
def GBDT_strain(df, hyperparams, variables, param):
    list_ip_series=[]
    strain_counts={}

    # Trasform hyperparams type.
    hyperparams = trans(hyperparams)

    for strain in sorted(list(set(df["Gene"]))):
        strains = df[df["Gene"]==strain].dropna(subset=[param])
        hyperparams_strain = hyperparams[strain]

        if len(strains)!=0:
            df_in = strains[variables]
            df_out = strains[param]

            # Perform GBDT.
            cfr = GradientBoostingRegressor(**hyperparams_strain)
            clf = cfr.fit(df_in, df_out)

            #Calculate feature importances.
            ip_series = pd.Series({n:s for n, s in zip(variables, clf.feature_importances_)},name=strain)
            list_ip_series.append(ip_series)
            
    feature_importances = pd.concat(list_ip_series,axis=1)

    return feature_importances

#Transform types.
def trans(hyperparams):
    trans_hyperparams = {gene:{
        "learning_rate":hyperparams.at["learning_rate",gene],
        "max_depth":int(hyperparams.at["max_depth",gene]),
        "n_estimators":int(hyperparams.at["n_estimators",gene]),
        "random_state":int(hyperparams.at["random_state",gene])
        } for gene in hyperparams.columns}

    return trans_hyperparams

In [8]:
# Import hyperparams.
hyperparams = load_csv(os.getcwd()+"/hyperparams/best_hyperparams_all.csv")

# Set variables and parameter.
variables = data.columns[5:-2]
param = "K_ave"

In [9]:
# Perform GBDT.
feature_importances = GBDT_strain(data, hyperparams, variables, param)

In [10]:
feature_importances

WT      acpT      aphA      appA      bioA      bioB  \
Glucose      0.116614  0.105157  0.156274  0.193483  0.125786  0.236256   
Citrate      0.002789  0.004451  0.008108  0.002017  0.008506  0.019538   
Ammonium     0.017966  0.007804  0.055960  0.002752  0.004339  0.000358   
Phosphate    0.007206  0.052579  0.006117  0.028635  0.010279  0.060902   
Sulfate      0.007323  0.009309  0.013067  0.003633  0.010747  0.016854   
Thiosulfate  0.006445  0.008498  0.001228  0.006174  0.000186  0.005594   
Cl           0.023757  0.038924  0.022328  0.078132  0.034171  0.036622   
Ala          0.000352  0.000414  0.000000  0.001866  0.000291  0.000000   
Arg          0.000040  0.004479  0.000000  0.007492  0.000405  0.000000   
Asn          0.015172  0.004180  0.002227  0.002255  0.000868  0.001658   
Asp          0.012011  0.001840  0.007040  0.018273  0.000797  0.019179   
Cys          0.004751  0.006732  0.009378  0.007766  0.016140  0.004582   
Glu          0.000627  0.002435  0.000771  0.001278  0.007008  0.006364   
Gln          0.015947  0.008746  0.000533  0.007150  0.001962  0.000869   
Gly          0.000256  0.002242  0.000000  0.005789  0.000052  0.011267   
His          0.000166  0.008009  0.000000  0.001554  0.001464  0.000000   
Ile          0.182555  0.094976  0.092209  0.088637  0.118918  0.103730   
Leu          0.006616  0.001755  0.002796  0.000443  0.008697  0.000000   
Lys          0.001746  0.001204  0.000000  0.004550  0.005223  0.009785   
Met          0.003196  0.004533  0.000000  0.001924  0.001348  0.000000   
Phe          0.000389  0.001984  0.000000  0.004623  0.001090  0.000000   
Pro          0.011349  0.002047  0.007040  0.006933  0.001067  0.006255   
Ser          0.001398  0.005675  0.000000  0.011320  0.000146  0.002299   
Thr          0.013728  0.003385  0.013230  0.008528  0.019099  0.001047   
Trp          0.003200  0.000428  0.003869  0.003197  0.002692  0.002695   
Tyr          0.002839  0.002559  0.001955  0.001628  0.004671  0.006367   
Val          0.174950  0.253734  0.232250  0.209694  0.213315  0.145319   
K            0.008820  0.012940  0.024119  0.051351  0.103065  0.086050   
Na           0.022671  0.032200  0.116445  0.008699  0.018788  0.014511   
Mg           0.034203  0.076122  0.071084  0.071304  0.083054  0.085431   
Ca           0.012307  0.016827  0.000000  0.003107  0.018158  0.002472   
Fe           0.045898  0.043386  0.049298  0.075590  0.039317  0.005704   
Cu           0.007079  0.014271  0.014061  0.004186  0.025204  0.020793   
Zn           0.018329  0.039983  0.001439  0.011544  0.003075  0.001123   
Molybdate    0.015955  0.012573  0.003361  0.010287  0.007545  0.000000   
VB1          0.001924  0.011702  0.006861  0.004405  0.005344  0.000000   
VB2          0.064703  0.071693  0.051255  0.024813  0.066882  0.038607   
VB3          0.005802  0.000451  0.000969  0.003212  0.005423  0.002176   
VB5          0.004558  0.001537  0.000000  0.008141  0.000772  0.003500   
VB6          0.002378  0.000680  0.000938  0.003644  0.002175  0.018923   
VB8          0.001770  0.004426  0.000920  0.002805  0.002512  0.000000   
VB9          0.001575  0.009624  0.002828  0.001772  0.000137  0.000000   
VB12         0.022935  0.001821  0.000000  0.001619  0.004222  0.000000   
Borate       0.012580  0.009557  0.000000  0.001420  0.007570  0.005544   
PABA         0.083123  0.002126  0.020040  0.002378  0.007492  0.017627   

                 bioC      bioD      bioF      bioH  ...      ygaD      ygcF  \
Glucose      0.126782  0.235474  0.163699  0.255094  ...  0.172910  0.249212   
Citrate      0.000513  0.023892  0.023087  0.022405  ...  0.014982  0.007929   
Ammonium     0.004912  0.028141  0.002321  0.041871  ...  0.005896  0.009557   
Phosphate    0.011247  0.003056  0.019315  0.018719  ...  0.005121  0.055483   
Sulfate      0.000938  0.007247  0.008954  0.044037  ...  0.005208  0.008622   
Thiosulfate  0.011696  0.002233  0.000371  0.000399  ...  0.040064  0.001469   
Cl      

# Perform GBDT and SHAP for the K_sd dataset.

In [11]:
def make_unique_data(df):
    # Extract unique conditions.
    df_unique = df[~df["Condition ID_unique"].duplicated()].set_index("Condition ID_unique").loc[:,"Glucose":"PABA"]
    
    # Calculate standard deviation for each condition.
    df_unique["K_std_cond"] = df.groupby("Condition ID_unique")["K_ave"].std() 

    return df_unique

In [12]:
# Perform SHAP.
def shap_analysis(df, hyperparams, variables, param):
    df_in = df[variables]
    df_out = df[param]
    
    def GBDT(df_in, df_out, hyperparams, variables, param):
        hyperparams = trans(hyperparams)["K_std"]
    
        cfr = GradientBoostingRegressor(**hyperparams)
        clf = cfr.fit(df_in, df_out)
    
        return clf

    # Perform GBDT.
    clf = GBDT(df_in, df_out, hyperparams, variables, param)

    # Perform SHAP.
    shap.initjs()
    explainer = shap.Explainer(clf,df_in)
    shap_values = explainer(df_in)

    
    return shap_values.values

In [13]:
# Make unique data.
unique_data = make_unique_data(data)

In [14]:
unique_data

Glucose   Citrate  Ammonium  Phosphate   Sulfate  \
Condition ID_unique                                                      
C00L2-1              0.376577 -2.744727  0.514548   1.820595  0.209431   
C00L3-1              1.049218 -1.045757  1.217484   2.140099  0.812608   
C0M63-1              1.342423 -4.744727  1.477121   2.000000  1.181895   
C0001-1              1.049218  0.255273  1.079181   1.926265  0.781237   
C0010-1              0.342423 -2.744727  1.079181   2.245747  0.810980   
...                       ...       ...       ...        ...       ...   
C0134-1              2.260548  0.255273  0.502427   2.100405  0.314010   
C0135-1              2.260548 -4.744727  2.262451   2.056200  1.963862   
C0136-1              2.260548  0.255273  0.514548   2.237343  0.209898   
C0137-1              2.260548 -1.045757  1.217484   1.649821  0.811553   
C0138-1              2.260548 -2.744727  1.968483   2.202856  1.022988   

                     Thiosulfate        Cl       Ala       Arg       Asn  ...  \
Condition ID_unique                                                       ...   
C00L2-1                -1.346787 -0.831223 -2.744727 -2.744727 -2.744727  ...   
C00L3-1                 0.352183  0.863397 -1.045757 -1.045757 -1.045757  ...   
C0M63-1                -3.346787 -1.823909 -4.744727 -4.744727 -4.744727  ...   
C0001-1                 1.653213  0.670720  0.255273 -1.045757 -4.744727  ...   
C0010-1                 0.352183  0.563848 -4.744727 -4.744727 -1.045757  ...   
...                          ...       ...       ...       ...       ...  ...   
C0134-1                -3.346787  0.640312 -2.744727 -1.045757 -4.744727  ...   
C0135-1                -3.346787  1.260103 -4.744727 -4.744727 -2.744727  ...   
C0136-1                 0.352183  1.041467 -1.045757 -2.744727  0.255273  ...   
C0137-1                 1.653213  1.487283 -2.744727 -1.045757 -1.045757  ...   
C0138-1                -1.346787  1.970965  0.255273  0.255273 -4.744727  ...   

                          VB2       VB3       VB5       VB6       VB8  \
Condition ID_unique                                                     
C00L2-1             -4.443697 -3.267606 -3.899629 -4.829856 -5.899629   
C00L3-1             -2.744727 -1.568636 -2.200659 -3.130886 -4.200659   
C0M63-1             -6.443697 -5.267606 -5.899629 -6.744727 -7.899629   
C0001-1             -2.744727 -3.267606 -0.899629 -4.829856 -4.200659   
C0010-1             -6.443697 -1.568636 -3.899629 -1.829856 -4.200659   
...                       ...       ...       ...       ...       ...   
C0134-1             -4.443697 -3.267606 -0.899629 -6.744727 -7.899629   
C0135-1             -2.744727 -1.568636 -2.200659 -4.829856 -4.200659   
C0136-1             -4.443697 -5.267606 -3.899629 -1.829856 -5.899629   
C0137-1             -1.443697 -0.267606 -0.899629 -3.130886 -2.899629   
C0138-1             -6.443697 -3.267606 -5.899629 -6.744727 -7.899629   

                          VB9      VB12    Borate      PABA  K_std_cond  
Condition ID_unique                                                      
C00L2-1             -6.443697 -7.744727 -3.142668 -3.267606    0.023899  
C00L3-1             -4.744727 -6.045757 -1.443697 -1.568636    0.063555  
C0M63-1             -8.443697 -9.744727 -5.142668 -5.267606    0.061187  
C0001-1             -8.443697 -6.045757 -1.443697 -0.267606    0.016484  
C0010-1             -4.744727 -4.744727 -5.142668 -3.267606    0.026598  
...                       ...       ...       ...       ...         ...  
C0134-1             -8.443697 -4.744727 -5.142668 -5.267606    0.054522  
C0135-1             -8.443697 -7.744727 -3.142668 -3.267606    0.003521  
C0136-1             -4.744727 -9.744727 -0.142668 -1.568636    0.037944  
C0137-1             -6.443697 -4.744727 -5.142668 -5.267606    0.016828  
C0138-1             -3.443697 -6.045757 -1.443697 -0.267606    0.109466  

[138 rows x 46 columns]

In [15]:
# Import hyperparams.
hyperparams_Kstd = load_csv(os.getcwd()+"/hyperparams/best_hyperparams_Kstd.csv")

# Set variables and parameter.
variables = data.columns[5:-2]
param = "K_std_cond"

In [16]:
shap_analysis(unique_data, hyperparams_Kstd, variables, param)

array([[-5.74953661e-03,  0.00000000e+00,  9.34111643e-05, ...,
        -4.39727029e-04,  0.00000000e+00,  0.00000000e+00],
       [ 4.45242197e-03,  0.00000000e+00, -6.75893663e-04, ...,
         4.46199893e-04,  0.00000000e+00,  0.00000000e+00],
       [ 4.13254160e-03,  0.00000000e+00, -1.47879942e-03, ...,
        -6.24182998e-04,  0.00000000e+00,  0.00000000e+00],
       ...,
       [ 2.76820284e-03,  0.00000000e+00,  1.69220677e-04, ...,
        -2.00868302e-04,  0.00000000e+00,  0.00000000e+00],
       [ 4.45242197e-03,  0.00000000e+00, -1.75404599e-03, ...,
         4.46199893e-04,  0.00000000e+00,  0.00000000e+00],
       [ 4.45242197e-03,  0.00000000e+00, -6.75893663e-04, ...,
         7.64571230e-04,  0.00000000e+00,  0.00000000e+00]])

# Perform GBDT for the data sets divided by glucose concentration.

In [17]:
def divide_dataset(df):
    #Divide the dataset by glucose concentration.
    df_glcL = df[df["Glucose"]<0.7]
    df_glcM = df[(df["Glucose"]>=0.7)&(df["Glucose"]<1.7)]
    df_glcH = df[df["Glucose"]>=1.7]

    return df_glcL, df_glcM, df_glcH

In [18]:
# Make divided datasets.
data_glcL, data_glcM, data_glcH = divide_dataset(data)

# Import hyperparams.
hyperparams_glcL = load_csv(os.getcwd()+"/hyperparams/best_hyperparams_glcL.csv")
hyperparams_glcM = load_csv(os.getcwd()+"/hyperparams/best_hyperparams_glcM.csv")
hyperparams_glcH = load_csv(os.getcwd()+"/hyperparams/best_hyperparams_glcH.csv")

# Set variables and parameter.
variables = data.columns[5:-2]
param = "K_ave"

In [19]:
# Perform GBDT.
feature_importances_glcL = GBDT_strain(data_glcL, hyperparams_glcL, variables, param)
feature_importances_glcM = GBDT_strain(data_glcM, hyperparams_glcM, variables, param)
feature_importances_glcH = GBDT_strain(data_glcH, hyperparams_glcH, variables, param)

In [20]:
feature_importances_glcL

WT      acpT      aphA      appA      bioA      bioB  \
Glucose      8.722870e-07  0.000000  0.000000  0.000148  0.007855  0.000080   
Citrate      4.233969e-03  0.040059  0.005032  0.004527  0.004673  0.010973   
Ammonium     3.490900e-02  0.000000  0.000895  0.005493  0.002838  0.028178   
Phosphate    6.543894e-02  0.053589  0.000000  0.000913  0.002465  0.016127   
Sulfate      3.208967e-02  0.000000  0.062982  0.028977  0.127994  0.009315   
Thiosulfate  1.484737e-04  0.000000  0.030788  0.000008  0.000085  0.020733   
Cl           1.715974e-02  0.015296  0.017620  0.057187  0.008199  0.181774   
Ala          1.484277e-04  0.004705  0.016453  0.005609  0.000291  0.013771   
Arg          1.258241e-03  0.000887  0.000000  0.002054  0.000576  0.001830   
Asn          1.321982e-02  0.010196  0.007851  0.000304  0.062980  0.016727   
Asp          7.384005e-03  0.034179  0.075015  0.067394  0.007846  0.038836   
Cys          5.050055e-04  0.000000  0.021057  0.034807  0.000474  0.023124   
Glu          6.494697e-03  0.000864  0.000000  0.000966  0.000458  0.006088   
Gln          3.085925e-03  0.007306  0.052517  0.013472  0.072601  0.009285   
Gly          2.097656e-03  0.000363  0.000000  0.005134  0.002339  0.000851   
His          3.581225e-05  0.000000  0.000000  0.000007  0.000642  0.000380   
Ile          2.072478e-01  0.222336  0.166220  0.189045  0.218947  0.131246   
Leu          1.603451e-02  0.000780  0.065285  0.006529  0.000605  0.005795   
Lys          1.865466e-05  0.001883  0.000000  0.000580  0.003133  0.018682   
Met          1.394017e-04  0.000000  0.000000  0.000002  0.012642  0.001134   
Phe          1.888471e-03  0.000648  0.002307  0.001786  0.004334  0.002198   
Pro          5.366468e-03  0.002216  0.050438  0.030325  0.004157  0.009293   
Ser          6.408339e-03  0.002005  0.005424  0.016066  0.004390  0.007795   
Thr          1.272532e-02  0.011441  0.006065  0.008009  0.005499  0.002967   
Trp          3.217095e-04  0.026401  0.001021  0.002059  0.000371  0.001663   
Tyr          4.476802e-03  0.000000  0.000658  0.011880  0.000018  0.002316   
Val          2.032916e-01  0.235099  0.196425  0.192760  0.234337  0.160893   
K            3.405779e-02  0.000320  0.003275  0.036808  0.030887  0.003259   
Na           1.034144e-01  0.023171  0.038214  0.024741  0.024221  0.050679   
Mg           3.408421e-04  0.001586  0.005134  0.005769  0.007054  0.007440   
Ca           8.611316e-02  0.157652  0.023570  0.037965  0.089244  0.033794   
Fe           7.570523e-03  0.010898  0.044095  0.055238  0.007456  0.024421   
Cu           1.577790e-03  0.002399  0.015541  0.000007  0.002116  0.003149   
Zn           6.372716e-03  0.040426  0.013528  0.021169  0.003530  0.021364   
Molybdate    3.423293e-03  0.000000  0.000000  0.027004  0.006578  0.002333   
VB1          8.929883e-07  0.000631  0.008806  0.011359  0.000091  0.000417   
VB2          9.532341e-02  0.074612  0.015438  0.000658  0.014227  0.004656   
VB3          1.573129e-05  0.002413  0.000000  0.030151  0.007494  0.002875   
VB5          3.142340e-03  0.000000  0.001553  0.032003  0.013107  0.102914   
VB6          3.948428e-03  0.000315  0.042760  0.006799  0.001327  0.006911   
VB8          2.055745e-03  0.000000  0.000000  0.021860  0.001114  0.004762   
VB9          9.560337e-04  0.009154  0.004030  0.000691  0.000230  0.000059   
VB12         6.214630e-04  0.003380  0.000000  0.000008  0.000038  0.001763   
Borate       2.376132e-04  0.002788  0.000000  0.001554  0.000364  0.000798   
PABA         4.698473e-03  0.000000  0.000000  0.000174  0.000172  0.006353   

                     bioC          bioD      bioF      bioH  ...      ygaD  \
Glucose      1.789379e-06  1.935815e-05  0.001322  0.000260  ...  0.019101   
Citrate      2.998230e-02  1.776944e-03  0.006365  0.069605  ...  0.016568   
Ammonium     8.180791e-03  2.157967e-02  0.001578  0.125781  ...  0.030368   
Phosphate    1.842722e-02  5.691733e-07  0.001555  0.013921  ...  0.066265  

In [21]:
feature_importances_glcM

WT          acpT      aphA      appA          bioA  \
Glucose      6.631102e-07  0.000000e+00  0.003622  0.201114  5.962207e-05   
Citrate      1.043417e-03  1.942876e-06  0.000105  0.001693  3.047586e-06   
Ammonium     3.738124e-01  7.696072e-02  0.004473  0.076300  3.526463e-02   
Phosphate    5.199247e-02  3.105619e-04  0.027285  0.067392  2.681609e-03   
Sulfate      2.220573e-01  2.136756e-03  0.000379  0.008712  7.402517e-03   
Thiosulfate  4.396159e-06  3.693971e-03  0.000154  0.001075  1.127901e-09   
Cl           3.360359e-03  8.457539e-02  0.082793  0.060733  3.108683e-02   
Ala          9.508676e-03  1.141136e-04  0.001896  0.002393  1.954334e-01   
Arg          1.719419e-02  6.211481e-06  0.000332  0.004879  6.660993e-02   
Asn          1.149765e-09  2.282111e-04  0.000096  0.000395  1.079700e-04   
Asp          7.318788e-03  1.416836e-06  0.003072  0.016631  5.331154e-08   
Cys          1.112564e-04  1.507907e-02  0.103728  0.001054  1.831374e-06   
Glu          5.389918e-05  1.830823e-09  0.000181  0.008700  7.882263e-04   
Gln          1.281827e-01  2.474428e-02  0.016119  0.072603  7.683529e-03   
Gly          3.427564e-04  3.561690e-02  0.053948  0.022601  2.445989e-03   
His          4.464769e-04  1.436012e-07  0.000712  0.010458  0.000000e+00   
Ile          2.726049e-02  1.624379e-02  0.001037  0.003665  1.316685e-04   
Leu          2.031603e-05  8.234966e-04  0.006460  0.000002  3.101111e-09   
Lys          6.216510e-05  2.894392e-05  0.066267  0.000405  5.686670e-04   
Met          2.376132e-06  6.234724e-02  0.034028  0.004951  1.470356e-05   
Phe          1.379876e-04  3.778720e-05  0.000061  0.005422  6.452806e-05   
Pro          1.337480e-03  1.832224e-04  0.000269  0.000066  1.404557e-04   
Ser          9.169014e-03  1.714619e-04  0.000221  0.009542  1.582241e-01   
Thr          3.481798e-02  2.771703e-04  0.005368  0.000048  2.019825e-04   
Trp          2.354810e-04  6.253419e-03  0.000344  0.002124  5.405500e-04   
Tyr          1.250969e-12  1.498716e-06  0.000041  0.000084  5.819591e-04   
Val          1.074925e-03  3.844359e-01  0.119441  0.043546  1.532296e-02   
K            2.664124e-04  1.139263e-01  0.007724  0.005845  3.295395e-02   
Na           5.243714e-04  5.825669e-03  0.268765  0.011598  2.406757e-03   
Mg           4.229696e-03  2.657233e-04  0.005881  0.082532  4.694284e-03   
Ca           2.840156e-03  8.387126e-07  0.000295  0.013141  5.222117e-02   
Fe           2.507219e-02  1.068653e-01  0.024952  0.204602  1.430313e-01   
Cu           2.272877e-03  7.226108e-03  0.000854  0.016555  1.916531e-03   
Zn           1.371124e-04  2.310726e-04  0.007073  0.001817  1.523293e-06   
Molybdate    3.816612e-04  7.436388e-03  0.000028  0.012912  5.376646e-05   
VB1          8.611418e-03  3.450085e-02  0.001728  0.002046  3.616414e-03   
VB2          1.900129e-02  2.417825e-04  0.014754  0.011158  6.615920e-04   
VB3          1.022618e-02  6.518338e-06  0.000086  0.000324  2.544753e-03   
VB5          5.583406e-03  3.779098e-03  0.000558  0.000156  3.042479e-03   
VB6          2.740149e-03  3.334699e-04  0.028769  0.006520  2.227242e-01   
VB8          6.647867e-04  1.058233e-03  0.005390  0.000324  2.754528e-04   
VB9          2.071680e-03  6.931238e-04  0.006601  0.000962  3.927099e-04   
VB12         9.097497e-03  1.581337e-03  0.004239  0.000553  4.092519e-03   
Borate       1.707314e-06  3.098222e-06  0.089493  0.001065  9.708900e-06   
PABA         1.672939e-02  1.751538e-03  0.000376  0.001304  1.210293e-07   

                     bioB      bioC      bioD          bioF      bioH  ...  \
Glucose      4.344536e-07  0.000016  0.000075  2.068046e-06  0.053174  ...   
Citrate      1.834935e-04  0.004544  0.004349  3.442116e-04  0.001767  ...   
Ammonium     3.306202e-01  0.043221  0.261051  2.246958e-01  0.180234  ...   
Phosphate    1.366684e-02  0.000074  0.028284  2.421409e-05  0.077185  ...   
Sulfate      6.122152e-07  0.000061  0.005351  8.949435e-04  0.009521  ...   
Thiosulfate 

In [22]:
feature_importances_glcH

WT          acpT          aphA          appA          bioA  \
Glucose      0.000000  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
Citrate      0.000093  2.689149e-02  4.711763e-06  5.051153e-02  1.414951e-02   
Ammonium     0.068507  3.226206e-04  1.977405e-03  4.432492e-02  4.309697e-06   
Phosphate    0.008144  5.007073e-03  4.654749e-03  8.093879e-02  3.017343e-04   
Sulfate      0.001129  7.580991e-02  2.999929e-02  1.872120e-03  4.879985e-02   
Thiosulfate  0.004703  2.954389e-05  2.154161e-03  5.191789e-02  2.462415e-03   
Cl           0.013532  2.722021e-02  2.331135e-02  6.766850e-02  6.712345e-05   
Ala          0.000058  3.652389e-13  2.770939e-07  2.121434e-12  4.041575e-03   
Arg          0.001100  5.443833e-04  2.404743e-03  2.321440e-03  4.044590e-05   
Asn          0.000076  1.885264e-02  6.693483e-06  4.284751e-03  1.663423e-05   
Asp          0.003548  7.231966e-03  1.640418e-05  1.412152e-02  7.202099e-06   
Cys          0.000000  5.644730e-14  1.244892e-03  7.226581e-05  4.125234e-03   
Glu          0.000000  2.837935e-02  1.322334e-09  1.814843e-03  4.958730e-03   
Gln          0.019505  1.418764e-01  1.883313e-04  5.270239e-05  2.710810e-06   
Gly          0.011042  1.982219e-05  3.092228e-02  1.231787e-03  1.883845e-04   
His          0.001001  9.414907e-04  4.040957e-04  1.477499e-02  1.950613e-04   
Ile          0.034240  2.685952e-04  1.766849e-04  4.331206e-02  1.104055e-02   
Leu          0.000240  8.119299e-04  1.860215e-06  5.900668e-02  3.394953e-02   
Lys          0.009272  3.609305e-06  9.016160e-02  1.497504e-03  1.135368e-02   
Met          0.002126  2.420126e-02  0.000000e+00  4.487099e-03  1.356242e-02   
Phe          0.027776  1.258034e-12  2.049972e-03  1.862568e-03  8.692674e-06   
Pro          0.080119  1.690730e-03  5.721273e-05  4.271711e-03  8.886493e-04   
Ser          0.019174  3.026582e-05  1.369114e-03  2.517505e-02  1.179728e-02   
Thr          0.021996  1.082770e-05  3.392488e-02  1.460837e-04  2.248350e-04   
Trp          0.035951  1.098719e-04  1.214309e-01  7.574494e-04  6.298258e-02   
Tyr          0.000000  7.457660e-07  4.602224e-03  2.405846e-03  2.502802e-01   
Val          0.017213  1.425937e-01  1.335289e-05  7.094084e-02  2.166896e-01   
K            0.092330  5.602353e-02  7.808350e-02  3.950538e-02  7.259704e-02   
Na           0.067513  1.246990e-01  1.718112e-03  6.127998e-02  1.431450e-02   
Mg           0.047533  5.152845e-02  4.893148e-02  3.971894e-02  1.297870e-01   
Ca           0.041013  6.830126e-04  6.274612e-02  6.860319e-04  7.159614e-04   
Fe           0.004894  2.096941e-02  1.348208e-08  4.402366e-05  1.536725e-04   
Cu           0.018769  1.551294e-02  8.392997e-03  5.689965e-04  2.736842e-08   
Zn           0.002109  4.292493e-03  3.724217e-03  1.883745e-03  2.003318e-05   
Molybdate    0.009413  7.332696e-02  2.577624e-03  7.216553e-06  3.144598e-02   
VB1          0.013116  7.673656e-02  4.490658e-02  3.572634e-03  2.177290e-03   
VB2          0.103572  1.236827e-03  5.177821e-06  5.358972e-02  6.084964e-03   
VB3          0.002585  2.130820e-03  5.032421e-03  8.592651e-06  2.519304e-04   
VB5          0.081382  1.354624e-03  9.010230e-02  3.307472e-02  3.994804e-03   
VB6          0.000000  1.935967e-03  3.462869e-02  4.447191e-06  2.021132e-04   
VB8          0.000000  2.455823e-03  0.000000e+00  1.762013e-02  3.347660e-06   
VB9          0.125600  1.655147e-03  2.665659e-01  1.345816e-01  2.090544e-05   
VB12         0.000686  1.735307e-03  3.016073e-07  1.453780e-06  4.583287e-02   
Borate       0.008944  6.087463e-02  1.492660e-03  6.406690e-02  1.587343e-04   
PABA         0.000000  1.271250e-08  1.471650e-05  1.459632e-05  9.993155e-05   

                 bioB          bioC          bioD          bioF          bioH  \
Glucose      0.000000  0.000000e+00  0.000000e+00  1.636107e-05  0.000000e+00   
Citrate      0.121818  6.737744e-04  2.003468e-02  2.099292e-05  9.033393e-03   
Ammonium     0.000330  0.000000e+00  4.356206e-03